# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Program: Computer Systems Engineering  </center>
---
### <center> **Spring 2025** </center>
---

**Lab 07**: Structured Streaming with Files

**Date**: 31/03/2025

**Team Name**: Par De Dos

**Professor**: Pablo Camarillo Ramirez